In [1]:
pip install flask flask-cors

Note: you may need to restart the kernel to use updated packages.


In [1]:
from flask import Flask
import threading

app = Flask(__name__)

@app.route('/')
def home():
    return "Hello, Flask from Jupyter Notebook!"

# Start Flask server in a separate thread
def run_app():
    app.run(debug=True, use_reloader=False)  # Disable reloader to prevent threading issues

threading.Thread(target=run_app).start()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


In [3]:
from flask import Flask, request, jsonify
from flask_cors import CORS
from flask_cors import cross_origin
import sqlite3
import csv
import json

app = Flask(__name__)
CORS(app)  # Enable CORS for cross-origin requests

# Connect to SQLite database
DB_FILE = "books_inventory1.db"

def connect_db():
    conn = sqlite3.connect(DB_FILE)
    return conn

# Initialize the database
def init_db():
    conn = connect_db()
    cursor = conn.cursor()
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS Inventory (
        entry_id INTEGER PRIMARY KEY AUTOINCREMENT,
        title TEXT NOT NULL,
        author TEXT NOT NULL,
        genre TEXT NOT NULL,
        publication_date DATE,
        isbn TEXT UNIQUE NOT NULL
    )
    ''')
    conn.commit()
    conn.close()

# Route: Add a new book
@app.route('/add-book', methods=['POST'])
@cross_origin()
def add_book():
    data = request.json
    try:
        conn = connect_db()
        cursor = conn.cursor()
        cursor.execute('''
        INSERT INTO Inventory (title, author, genre, publication_date, isbn)
        VALUES (?, ?, ?, ?, ?)
        ''', (data['title'], data['author'], data['genre'], data['publication_date'], data['isbn']))
        conn.commit()
        conn.close()
        return jsonify({"message": "Book added successfully!"}), 201
    except sqlite3.IntegrityError as e:
        return jsonify({"error": str(e)}), 400
    pass

# Route: Filter books
@app.route('/filter-books', methods=['GET'])
def filter_books():
    title = request.args.get('title', '')
    author = request.args.get('author', '')
    genre = request.args.get('genre', '')
    date_from = request.args.get('date_from', '')
    date_to = request.args.get('date_to', '')

    query = "SELECT * FROM Inventory WHERE 1=1"
    params = []

    if title:
        query += " AND title LIKE ?"
        params.append(f"%{title}%")
    if author:
        query += " AND author LIKE ?"
        params.append(f"%{author}%")
    if genre:
        query += " AND genre = ?"
        params.append(genre)
    if date_from:
        query += " AND publication_date >= ?"
        params.append(date_from)
    if date_to:
        query += " AND publication_date <= ?"
        params.append(date_to)

    conn = connect_db()
    cursor = conn.cursor()
    cursor.execute(query, params)
    rows = cursor.fetchall()
    conn.close()

    columns = ['entry_id', 'title', 'author', 'genre', 'publication_date', 'isbn']
    books = [dict(zip(columns, row)) for row in rows]
    return jsonify(books)

# Route to get all books
@app.route('/books', methods=['GET'])
def get_books():
    return jsonify(books)

# Route: Export books to CSV
@app.route('/export-csv', methods=['GET'])
def export_csv():
    conn = connect_db()
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM Inventory")
    rows = cursor.fetchall()
    conn.close()

    file_name = "books_inventory1.csv"
    with open(file_name, "w", newline="") as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['entry_id', 'title', 'author', 'genre', 'publication_date', 'isbn'])
        writer.writerows(rows)

    return jsonify({"message": f"Data exported to {file_name}!"})

# Route: Export books to JSON
@app.route('/export-json', methods=['GET'])
def export_json():
    conn = connect_db()
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM Inventory")
    rows = cursor.fetchall()
    conn.close()

    columns = ['entry_id', 'title', 'author', 'genre', 'publication_date', 'isbn']
    books = [dict(zip(columns, row)) for row in rows]

    file_name = "books_inventory1.json"
    with open(file_name, "w") as jsonfile:
        json.dump(books, jsonfile, indent=4)

    return jsonify({"message": f"Data exported to {file_name}!"})

if __name__ == '__main__':
    init_db()
    app.run(debug=True)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Restarting with windowsapi reloader


SystemExit: 1

127.0.0.1 - - [15/Nov/2024 13:02:31] "OPTIONS /books HTTP/1.1" 404 -
127.0.0.1 - - [15/Nov/2024 13:02:56] "GET /books HTTP/1.1" 404 -
